In [7]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler

# Carregar os dados
# df = pd.read_csv('Reddit_Data.csv')
# df_filtrado = df[df['category'] != 0]
# df_1 = df_filtrado['clean_comment']
# rotulos = df_filtrado['category']

data = pd.read_csv('new_comentarios_ingles.csv')
df_1 = data['Comentario']
rotulos = data['sentimento']

df['Comentario'] = df['Comentario'].apply(lambda x: x.lower())

# Definir polarizações de sentimento
def define_sentiment(rating):
    if rating == 'positivo':
        return 1  
    elif rating == 'neutro':
        return 2
    else:
        return 0 

data['sentimento_polarizado'] = data['sentimento'].apply(define_sentiment)

rotulos = data['sentimento_polarizado']

# Tokenização e lematização
comentarios_tokenizados = []
wnl = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [wnl.lemmatize(plv) for plv in tokens]

for tok in df_1:
    tokens = nltk.word_tokenize(str(tok))
    comentarios_tokenizados.append(tokens)

lemm_tks = [lemmatize_tokens(tokens) for tokens in comentarios_tokenizados]

# Divisão em conjuntos de treinamento e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(lemm_tks, rotulos, test_size=0.2, random_state=42, stratify=rotulos)


# Vetorização dos dados de texto
vectorizer = CountVectorizer()
X_treino_vect = vectorizer.fit_transform([' '.join(comentario) for comentario in X_treino])
X_teste_vect = vectorizer.transform([' '.join(comentario) for comentario in X_teste])

# Aplicando Random Oversampling
oversampler = RandomOverSampler()
X_treino_res, y_treino_res = oversampler.fit_resample(X_treino_vect, y_treino)

# Treinamento do classificador de Regressão Logística
classifier = LogisticRegression(max_iter=1000)

# Validação cruzada
scores = cross_val_score(classifier, X_treino_res, y_treino_res, cv=5)  # cv=5 indica 5-fold cross-validation

# Treinamento final do modelo
classifier.fit(X_treino_res, y_treino_res)

# Predição nos dados de teste
predicoes = classifier.predict(X_teste_vect)

# Calcular precisão, recall e F-score
precision = precision_score(y_teste, predicoes, average='weighted')
recall = recall_score(y_teste, predicoes, average='weighted')
f_score = f1_score(y_teste, predicoes, average='weighted')
acuracia = accuracy_score(y_teste, predicoes)
relatorio_classificacao = classification_report(y_teste, predicoes, target_names=['Negativo', 'Positivo', 'Neutro'])

# Imprimir resultados
print("Resultados da validação cruzada:")
print("Scores:", scores)
print("Precisão média:", scores.mean())
print("Acuracia: {:.2f}".format(acuracia))
print("Precisão: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F-score: {:.2f}".format(f_score))
print("\nRelatório de Classificação:")
print(relatorio_classificacao)

Resultados da validação cruzada:
Scores: [0.94339623 0.93803392 0.93675889 0.93484636 0.94071146]
Precisão média: 0.9387493715102279
Acuracia: 0.77
Precisão: 0.81
Recall: 0.77
F-score: 0.78

Relatório de Classificação:
              precision    recall  f1-score   support

    Negativo       0.23      0.29      0.26       220
    Positivo       0.90      0.85      0.87      3268
      Neutro       0.19      0.28      0.23       317

    accuracy                           0.77      3805
   macro avg       0.44      0.47      0.45      3805
weighted avg       0.81      0.77      0.78      3805



In [8]:
print(len(data))

19025
